In [ ]:
#imports

import numpy as np
import os
import matplotlib.pyplot as plt
from alt_period_finder import alt_period_finder
plt.style.use('seaborn-dark-palette')
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn
import butcher1 as butcher
from astropy.timeseries import LombScargle
from scipy.stats import norm
from scipy.optimize import curve_fit
from astropy import modeling
from tqdm import tqdm
from IPython.display import display, Math

import scipy.optimize as op
import os
import emcee
import corner
import time

%matplotlib inline

import emcee


font = {'family':'normal', 'weight':'bold', 'size':36}
plt.rc('font', **font)

In [ ]:
#reading in the data

#Get the data directory
cwd = os.getcwd()
data_dir = cwd.replace('Figure_6', 'Data\\')

#ASAS data
orgasas_data = ascii.read(data_dir + 'asas.csv')
asas_mask = (orgasas_data['mag'] > np.mean(orgasas_data['mag'])-2*np.std(orgasas_data['mag']))*(orgasas_data['emag'] < 0.05)
asas_data = orgasas_data[asas_mask]

asas_flux = butcher.mag_to_flux(asas_data['mag'])
asas_eflux = butcher.emag_to_eflux(asas_data['mag'], asas_data['emag'])

#ASASSN data
orgasassn_data = ascii.read(data_dir + 'asassn.csv')
asassn_mask = (orgasassn_data['mag'] > np.mean(orgasassn_data['mag'])-2*np.std(orgasassn_data['mag']))*(orgasassn_data['emag'] < 0.05)
asassn_data = orgasassn_data[asassn_mask]

asassn_flux = butcher.mag_to_flux(asassn_data['mag'])
asassn_eflux = butcher.emag_to_eflux(asassn_data['mag'], asassn_data['emag'])

#KELT data
orgkelt_data = ascii.read(data_dir + 'kelt.csv')
kelt_mask = (orgkelt_data['mag'] > np.mean(orgkelt_data['mag'])-2*np.std(orgkelt_data['mag']))*(orgkelt_data['emag'] < 0.05)
kelt_data = orgkelt_data[kelt_mask]

kelt_flux = butcher.mag_to_flux(kelt_data['mag'])
kelt_eflux = butcher.emag_to_eflux(kelt_data['mag'], kelt_data['emag'])

#PROMPT data
orgprompt_data = ascii.read(data_dir + 'prompt.csv') #time is JD-2450000
prompt_mask = (orgprompt_data['mag'] > np.mean(orgprompt_data['mag'])-2*np.std(orgprompt_data['mag']))*(orgprompt_data['emag'] < 0.05)
prompt_data = orgprompt_data[prompt_mask]

prompt_flux = butcher.mag_to_flux(prompt_data['mag'])
prompt_eflux = butcher.emag_to_eflux(prompt_data['mag'], prompt_data['emag'])

#ROAD data
orgroad_data = ascii.read(data_dir + 'road.csv') #time is JD-2450000
road_mask = (orgroad_data['mag'] > np.mean(orgroad_data['mag'])-2*np.std(orgroad_data['mag']))*(orgroad_data['emag'] < 0.05)
road_data = orgroad_data[road_mask]

road_flux = butcher.mag_to_flux(road_data['mag'])
road_eflux = butcher.emag_to_eflux(road_data['mag'], road_data['emag'])

In [ ]:
print(np.min(road_data['HJD']), np.max(road_data['HJD']), len(road_data))

In [ ]:
import astropy.units as u
def fit_sine(time, period, comb_sigma):
    
    def sine(x, period, a, b, d):
        #Creates a sine curve with parameters a, b, d but with fixed c (period)
        return a+b*np.sin((2*np.pi/period)*(x-d))
    
    #Fit a sine curve with this set period
    popt, pcov = curve_fit(sine, time, periods, p0 = [period, 3.2, 0.03, 56750], sigma=comb_sigma, absolute_sigma = True)
    return popt, pcov

def fit_sine2(time, period, fluxes, comb_sigma):
    
    def sine(x, period, a, b, d):
        #Creates a sine curve with parameters a, b, d but with fixed c (period)
        return a+b*np.sin((2*np.pi/period)*(x-d))
    
    #Fit a sine curve with this set period
    popt, pcov = curve_fit(sine, time, fluxes, p0 = [period, 3.2, 0.03, 56750], sigma=comb_sigma, absolute_sigma = True)
    return popt, pcov

def sine(x, period, a, b, d):
    #print(np.max((2*np.pi/period)*(x-d)))
    #print(np.min((2*np.pi/period)*(x-d)))
   
    return a+b*np.sin(((2*np.pi/period)*(x-d))*u.radian)

In [ ]:
#hmb_flux = butcher.long_correct(hmb_data['HJD'], hmb_flux, hmb_eflux)
asas_flux = butcher.long_correct(asas_data['MJD'], asas_flux, asas_eflux)
asassn_flux = butcher.long_correct(asassn_data['MJD'], asassn_flux, asassn_eflux)
kelt_flux = butcher.long_correct(kelt_data['HJD'], kelt_flux, kelt_eflux)
prompt_flux = butcher.long_correct(prompt_data['HJD'], prompt_flux, prompt_eflux)
road_flux = butcher.long_correct(road_data['HJD'], road_flux, road_eflux)

In [ ]:
def binner(bins, time, flux):
    means, errors = [], []
    midpoints = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
    for i in range(len(bins)-1):
        binned_mask = (time < bins[i+1])*(time > bins[i])
        means.append(np.mean(flux[binned_mask]))
        errors.append(np.std(flux[binned_mask])/np.sqrt(len(flux[binned_mask])))
    return means, errors, midpoints

In [ ]:
times = [asas_data['MJD'], asassn_data['MJD'], kelt_data['HJD'], prompt_data['HJD'], road_data['HJD']]
fluxes = [asas_flux, asassn_flux, kelt_flux, prompt_flux, road_flux]
uncertainties = [asas_eflux, asassn_eflux, kelt_eflux, prompt_eflux, road_eflux]
names = ['ASAS', 'ASAS-SN', 'KELT', 'PROMPT', 'ROAD']
periods=np.linspace(1.5,5, 500)

#fig, ax = plt.subplots(5, 2)

font = {'family':'normal', 'weight':'bold', 'size':8}
plt.rc('font', **font)


for i in range(5):
    yes = True
    if yes:
        pass
    else:
        
        time, flux, uncertainty = times[i], fluxes[i], uncertainties[i]
        frequency, power = LombScargle(time, flux-np.mean(flux), uncertainty).autopower(minimum_frequency=1/(np.max(periods)), maximum_frequency=1/(np.min(periods)), samples_per_peak=10)

        maximum_power = 1/np.max(frequency[np.where(power == np.max(power))[0]])

        ax[i, 0].plot(1/frequency, power, label = names[i] + ', Maximum Period: ' + '{0:.3f}'.format(maximum_power) + ' days')
        if i == 4:
            ax[i, 0].set_xlabel('Periodicity of the signal (Days)')
        ax[i, 0].set_ylabel('Power of the signal')
        ax[i, 0].axvline(x = 0.5*maximum_power, linestyle = '--', c = 'r', label = '0.5 times Maximum Period', alpha = 0.3)
        ax[i, 0].legend()

        means, errors, midpoints = binner(np.linspace(0, maximum_power, 50), time%(maximum_power), flux)
        ax[i, 1].errorbar(midpoints, means/np.mean(flux), errors, fmt = '.')

        popt, pcov = curve_fit(sine, midpoints, means/np.mean(flux), p0 = [maximum_power, 1, 0.03, 2.1], sigma=errors, absolute_sigma = True)
        print(np.diag(np.sqrt(pcov))[0])
        print('amp ', popt[2])

        sine_space = np.linspace(0, maximum_power*3, 500)

        ax[i, 1].plot(sine_space, sine(sine_space, popt[0], popt[1], popt[2], popt[3]), label = names[i] + ', Best sine with period: '+str(popt[0])[:5] + ' days')
        ax[i, 1].axhline(y = 1, linestyle = '--', c= 'grey', label = 'Mean')
        ax[i, 1].set_ylim(0.95, 1.05)
        ax[i, 1].legend()
        if i == 4:
            ax[i, 1].set_xlabel('Folded Period (Days)')
        ax[i, 1].set_ylabel('Mean Flux')

        print(np.min(time), np.max(time))
    
    
    
#fig = plt.gcf()
#fig.set_size_inches(10,10)
#plt.savefig('LombScargle_&_Sinefit.pdf')
#plt.show()

In [ ]:
#corr_hmb, p1 = butcher.short_correct(hmb_data['HJD'], hmb_flux, hmb_eflux)
corr_asas, p2 = butcher.short_correct(asas_data['MJD'], asas_flux, asas_eflux)
corr_asassn, p3 = butcher.short_correct(asassn_data['MJD'], asassn_flux, asassn_eflux)
corr_kelt, p4 = butcher.short_correct(kelt_data['HJD'], kelt_flux, kelt_eflux)
corr_prompt, p6 = butcher.short_correct(prompt_data['HJD'], prompt_flux, prompt_eflux)
corr_road, p5 = butcher.short_correct(road_data['HJD'], road_flux, road_eflux)

In [ ]:
#Adding the uncorrected data:
all_times = np.concatenate((np.array(asas_data['MJD']), np.array(asassn_data['MJD']), np.array(kelt_data['HJD']), np.array(prompt_data['HJD']), np.array(road_data['HJD']))) #np.array(hmb_data['HJD']),
all_flux = np.concatenate((np.array(asas_flux), np.array(asassn_flux), np.array(kelt_flux), np.array(prompt_flux), np.array(road_flux))) #np.array(hmb_flux),
all_eflux = np.concatenate((np.array(asas_eflux), np.array(asassn_eflux), np.array(kelt_eflux), np.array(prompt_eflux), np.array(road_eflux))) #np.array(hmb_eflux)

all_flux_short = np.concatenate((np.array(corr_asas), np.array(corr_asassn), np.array(corr_kelt), np.array(corr_prompt), np.array(corr_road))) #np.array(corr_hmb),
alt_flux_short, p555 = butcher.short_correct(all_times, all_flux, all_eflux)
print(alt_flux_short)
all_times_final, all_flux_final, all_flux_short_final, all_eflux_final = zip(*sorted(zip(all_times, all_flux, alt_flux_short, all_eflux)))

all_times_final = np.array(all_times_final)
all_flux_final = np.array(all_flux_final)
all_flux_short_final = np.array(all_flux_short_final)
all_eflux_final = np.array(all_eflux_final)
print(all_flux_short_final)
atf, aff, afsf, aef = np.zeros(len(all_times_final)), np.zeros(len(all_times_final)), np.zeros(len(all_times_final)), np.zeros(len(all_times_final))
atf[:], aff[:], afsf[:], aef[:] = all_times_final, all_flux_final, all_flux_short_final, all_eflux_final

print(len(atf))

#from numpy import savetxt
#savetxt('final_comb_data_S.csv', final_array, delimiter=',')
#savetxt('final_comb_flux.csv', flux_final, delimiter=',')
#savetxt('final_comb_eflux.csv', eflux_final, delimiter=',')


In [ ]:
#all_flux_short = np.concatenate((np.array(corr_hmb),np.array(corr_asas),np.array(corr_asassn),np.array(corr_kelt),np.array(corr_prompt),np.array(corr_road)))
print(all_times_final, all_flux_short_final, all_eflux_final)
#all_times = np.concatenate(all_times)
#all_eflux = np.concatenate(all_eflux)
plt.errorbar(all_times_final, all_flux_short_final, yerr=all_eflux_final, fmt='.', ms=3, elinewidth=0.5)
plt.show()

In [ ]:
#Seperate for each data set:
times = [asas_data['MJD'], asassn_data['MJD'], kelt_data['HJD'], prompt_data['HJD'], road_data['HJD']] #hmb_data['HJD'], 
fluxes = [asas_flux, asassn_flux, kelt_flux, prompt_flux, road_flux] #hmb_flux, 
efluxes = [asas_eflux, asassn_eflux, kelt_eflux, prompt_eflux, road_eflux] #hmb_eflux, 

sep_midpoints = []
sep_periods = []
sep_errors = []

sample_sizes = [75, 75, 75, 75, 75]
points_per_cycles = [2.5, 2.5, 2.5, 2.5, 2.5]

for i in range(5):
    alt_comb_midpoints = []
    alt_comb_periods = []
    alt_comb_errors = []
    
    midpoints, chunk_periods, errors = alt_period_finder(times[i], fluxes[i], efluxes[i], sample_size = sample_sizes[i], points_per_cycle = points_per_cycles[i])
    for k in chunk_periods:
        alt_comb_periods.append(k)
    for l in midpoints:
        alt_comb_midpoints.append(l)
    for m in errors:
        alt_comb_errors.append(m)
            
    alt_comb_midpoints = np.array(alt_comb_midpoints)
    alt_comb_periods = np.array(alt_comb_periods)
    alt_comb_errors = np.array(alt_comb_errors).astype(np.float)

    alt_better_mask = (alt_comb_periods < 3.29) * (alt_comb_periods > 3.11) * (alt_comb_errors < 0.05)
    
    true = 0
    for i in range(len(alt_better_mask)):
        if alt_better_mask[i] == True:
            true +=1
    print(true, len(alt_better_mask))
    sep_midpoints.append(alt_comb_midpoints[alt_better_mask])
    sep_periods.append(alt_comb_periods[alt_better_mask])
    sep_errors.append(alt_comb_errors[alt_better_mask])
    
combined_midpoints = np.concatenate(sep_midpoints)

In [ ]:
print(sep_midpoints)

names = ['ASAS', 'ASASSN', 'KELT', 'PROMPT', 'ROAD'] #'AAVSO', 
for i in range(5):
    if len(sep_midpoints[i]) != 0:#ASASSN does not yield any periods
        plt.errorbar(sep_midpoints[i], sep_periods[i], sep_errors[i], fmt='.')
    
figure = plt.gcf()
figure.set_size_inches(18, 12)
plt.xlim(np.min(combined_midpoints)-200, np.max(combined_midpoints) + 200)
#plt.ylim(3.1, 3.3)
#plt.savefig("75_days_3ppc_33result.pdf")
plt.show()


print(len(combined_midpoints))

In [ ]:
names = ['ASAS', 'ASASSN', 'KELT', 'PROMPT', 'ROAD'] #'AAVSO', 

fig, axs = plt.subplots(2, 1, sharex=True, tight_layout=True)

for i in range(5):
    if sep_midpoints[i] != []: #ASASSN does not yield any periods
        axs[1].errorbar(sep_midpoints[i], sep_periods[i], yerr=sep_errors[i], fmt='.', ms=5, elinewidth=0.5, label=names[i])

combined_midpoints = np.concatenate(sep_midpoints)
combined_periods = np.concatenate(sep_periods)
combined_errors = np.concatenate(sep_errors)

combined_masky = (combined_errors <  0.02) * (combined_midpoints > 57000)

average_period = np.average(sep_periods[-1], weights=sep_errors[-1])
print(average_period)
popt=fit_sine2(combined_midpoints[combined_masky], average_period, combined_periods[combined_masky], combined_errors[combined_masky])


axs[0].errorbar(all_times_final, all_flux_short_final, yerr=all_eflux_final, fmt='.', ms=3, elinewidth=0.5)
#axs[1].plot(np.linspace(np.min(combined_midpoints), np.max(combined_midpoints), 1000), sine(np.linspace(np.min(combined_midpoints), np.max(combined_midpoints), 1000), *popt), color='red', alpha=0.6, label='fit ['+str(popt[0])[:7]+"   "+str(popt[1])[:6]+"   "+str(popt[2])[:6]+']')
print(*popt)
figure = plt.gcf()
figure.set_size_inches(24, 18)
axs[1].axhline(average_period, color='black', alpha=0.5, ls='--')
axs[1].set_xlabel('Julian date -2400000 (days)')
axs[1].set_ylabel('Rotation period (days)')
axs[0].set_ylabel('Normalised Flux')
#plt.xlim(55000, 59000)
plt.legend()
#plt.savefig('lightcurve_and_variability.png')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=True, tight_layout=True)

mask6 = combined_periods < 3.17
bad_fit_mask = mask6#mask5 + mask1 + mask2 + mask3 
good_fit_mask = np.invert(bad_fit_mask)
names = ['ASAS', 'ASASSN', 'KELT', 'PROMPT', 'ROAD'] #'AAVSO', 


for i in range(5):
    if sep_midpoints[i] != []: #ASASSN does not yield any periods
        axs.errorbar(sep_midpoints[i], sep_periods[i], yerr=sep_errors[i], fmt='.', ms=3, elinewidth=0.5, label=names[i])
axs.axhline(average_period, color='black', alpha=0.3, ls='--')
axs.legend(fontsize=14, loc='upper left')

popt, pcov =fit_sine2(combined_midpoints[good_fit_mask], average_period, combined_periods[good_fit_mask], combined_errors[good_fit_mask])#combined_errors[good_fit_mask])
axs.plot(np.linspace(np.min(combined_midpoints[good_fit_mask]), np.max(combined_midpoints[good_fit_mask]), 1000), sine(np.linspace(np.min(combined_midpoints[good_fit_mask]), np.max(combined_midpoints[good_fit_mask]), 1000), *popt), color='red', zorder=20, alpha=0.6, label='fit ['+str(popt[0])[:7]+"   "+str(popt[1])[:6]+"   "+str(popt[2])[:6]+']')
#axs[1].errorbar(combined_midpoints[good_fit_mask], combined_periods[good_fit_mask], yerr=combined_errors[good_fit_mask], fmt='.', ms=3, elinewidth=0.5, label='Combined data used for fit')
axs.errorbar(combined_midpoints[bad_fit_mask], combined_periods[bad_fit_mask], yerr=combined_errors[bad_fit_mask], fmt='.', ms=2, elinewidth=0.5, color='black', alpha=1, label='Omitted data with periodic irregularities')
axs.axhline(average_period, color='black', alpha=0.3, ls='--')

print(*popt)
figure = plt.gcf()
figure.set_size_inches(18, 9)
axs.set_xlabel('MJD (days)')
axs.set_ylabel('Rotation period (days)')
axs.set_ylabel('Rotation period (days)')
#axs[0].set_ylabel('Normalised flux')
plt.legend(fontsize=14, loc='upper left')
plt.suptitle('Long term variability of J1407')
#plt.savefig('75_days_3ppc_33result_fit_with_prev_mask.pdf')
plt.show()

In [ ]:
x_data = combined_midpoints[good_fit_mask]
y_data = combined_periods[good_fit_mask]
y_err = combined_errors[good_fit_mask]

time_space = np.linspace(np.min(x_data)-100, np.max(x_data)+100, 5000)

x_data, y_data, y_err = zip(*sorted(zip(x_data, y_data, y_err)))
 
###par =    [2350,         55700,       3.2,      0.015] #Guess based on previous results
par =    [1950,         56800,       3.2,      0.018] #Guess based on previous results

labels = ["$P_{act}$", "$t_0$", "$P_{mean}$", "$a$"]

In [ ]:
def activity_model(t, P, t0, mean, a):
    P_activity = mean + a * np.sin(2*np.pi * (t-t0)/P) 
    return P_activity

In [ ]:
Pact_model = activity_model(time_space, 2350, 55700, 3.20, 0.015)

In [ ]:
def lnlike(theta, t, f, ferr):
    ''' theta holds the free parameters of the model, t,f, ferr are the noisy observed measurements'''
    P, t0, mean, a = theta

    # calculate the model with the values in theta, and the data in (t,f,ferr)
    model = activity_model(t, P, t0, mean, a)
    
    # calculate the chi squared for each epoch
    chi2 = np.power((f-model)/ferr,2.)
    
    # add up all the chi squareds, and the -0.5 is for emcee
    #print(-0.5*(np.sum(chi2)))
    return -0.5*(np.sum(chi2))


# this nll function is for the minimise function, which wants a MINIMISE a value, not maximise it.
nll = lambda *args: -lnlike(*args)

In [ ]:
result = op.minimize(nll, par, method='nelder-mead', args=(x_data,y_data,y_err),
                     options={'maxiter':10000,'xtol': 1e-8, 'disp': True})

print(result)

In [ ]:
# prior - here we choose whether to restrict the fitting paramaters
def lnprior(theta):
    P, t0, mean, a = theta
    if 500 < P < 3000 and 3.10 < mean < 3.3:
        return 0.0
    return -np.inf

In [ ]:
def lnprob(theta, t, f, ferr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, t, f, ferr)

In [ ]:
ndim, nwalkers = 4, 100

# we can add the parameters found in the LWFIT above:
par = result.x

pos = [par + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x_data,y_data,y_err), threads=4)

In [ ]:
#start = time.time()
pos, prob, state = sampler.run_mcmc(pos, 15000)
#end = time.time()
#print(end-start)

In [ ]:
print(sampler.get_autocorr_time(c=1))
print(sampler.acceptance_fraction)

In [ ]:
burn = 600
print(sampler.chain.shape)
samples = sampler.chain[:, burn:, :].reshape((-1, ndim))
print(samples.shape)

In [ ]:
for i in range(ndim):
    mcmc = np.percentile(samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.4f}_{{-{1:.4f}}}^{{{2:.4f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))

In [ ]:
inds = np.random.randint(len(samples), size=100)

font = {'family':'normal', 'weight':'normal', 'size':38}
plt.rc('font', **font)


fig1, f1a = plt.subplots(ncols=1,nrows=1, figsize=(10,6), constrained_layout=True)
colours = ['red', 'black', 'red', 'blue', 'blue', 'yellow']
odd_mask = combined_midpoints < 3.17

for i in range(5): #[ 2, 4]:
    if i > 1:
        f1a.errorbar(sep_midpoints[i],sep_periods[i],yerr=sep_errors[i],fmt='o',capsize=0 ,mew=3, elinewidth=1,ms=5, label=names[i])


a = np.percentile(samples[:, 0], [50])[0]
b = np.percentile(samples[:, 1], [50])[0]
c = np.percentile(samples[:, 2], [50])[0]
d = np.percentile(samples[:, 3], [50])[0]

a_copy = np.copy(a)
b_copy = np.copy(b)
c_copy = np.copy(c)
d_copy = np.copy(d)

    
Pact_model = activity_model(time_space, a, b, c, d)    
    
f1a.plot(time_space,Pact_model,'r-', c= 'brown', linewidth=3,zorder=10, label='Best fit')#fit ['+str(int(a+1))[:4]+"   "+str(c)[:5]+"   "+str(d)[:5]+']')
f1a.axhline(y= np.percentile(samples[:, 2], [50]), linestyle = '--', c='black')
#f1a.errorbar(combined_midpoints[bad_fit_mask], combined_periods[bad_fit_mask], yerr=combined_errors[bad_fit_mask], c = 'black', fmt='o', capsize=0 ,mew=3, elinewidth=1,ms=3, label='Omitted data')

f1a.set_xlabel('MJD (Days)')
f1a.set_ylabel('Measured rotational period (Days)')
for ind in inds:
    onesample = samples[ind]
    P, t0, mean, a = onesample

    # calculate the model with the values in theta, and the data in (t,f,ferr)
    onemodel = activity_model(time_space, P, t0, mean, a) 
    plt.plot(time_space, onemodel, "C1", alpha=0.1)
    if ind == inds[-1]:
        plt.plot(time_space, onemodel, "C1", alpha=0.2, label = 'Model fits')

figure = plt.gcf()
figure.set_size_inches(18, 12)
    
plt.legend(fontsize = 24)

#f1a.plot(time_space, linear(time_space, *lin_popt), lw = 3)

#fig1.savefig('Activity_cycle_30_05.pdf')

## BIC Test

In [ ]:
def bic(n, k, data, model, err):
    wlsq = np.sum((data-model)**2/err**2)
    return n*np.log(wlsq/n)+k*np.log(n)

In [ ]:
def linear(x, a, b):
    return a*x+b

In [ ]:
f_times, f_means, f_errs = np.concatenate(sep_midpoints), np.concatenate(sep_periods), np.concatenate(sep_errors)

lin_popt, pcov = curve_fit(linear, f_times, f_means, p0 = [0, 3.21], sigma=f_errs, absolute_sigma = True)
sine_popt = [a_copy, b_copy, c_copy, d_copy]

n = len(f_times)
k_sine = 4
k_lin = 2

sin_bic = bic(n, k_sine, f_means, activity_model(f_times, *sine_popt), f_errs)
lin_bic = bic(n, k_lin, f_means, linear(f_times, *lin_popt), f_errs)

print("the sinbic is: ", '{0:.2f}'.format(sin_bic))
print("the linbic is: ", '{0:.2f}'.format(lin_bic))